In [1]:
# to make the export go to the fitbit app, click on your icon, and order a google takeout 
# took like 8 hours the last time

In [2]:
import os
import pandas as pd

# Get the list of all files and directories
workingDataPath = "/home/chowder/Documents/workingData/fitbit/hr/"

pathOfExport = "/home/chowder/Documents/dataExports/fitbit/"
exportFilePath = "27-8-24/takeout-20240828T033834Z-001/Takeout/Fitbit/Global Export Data"

dir_list = os.listdir(pathOfExport + exportFilePath)

In [3]:
# read in the existing files and make the existing df
workingDataFiles = os.listdir(workingDataPath)
columnNames = ["sampleDT", "confidence", "value"]
dfSoFar = pd.DataFrame(columns=columnNames).set_index("sampleDT")

for dataFileName in workingDataFiles:
    dfSoFar = pd.concat([dfSoFar, pd.read_parquet(workingDataPath + dataFileName)]) 

dfSoFar = dfSoFar[~dfSoFar.index.duplicated(keep="first")].sort_index()

print(f"read in {len(dfSoFar)} number of rows from {len(workingDataFiles)} files")


read in 15826297 number of rows from 19 files


In [4]:
# get a list of unique dates in the index
#   removing the latest 3 days since they might be incomplete
datesSoFar = sorted(list(set(dfSoFar.index.date)))[:-3]
datesSoFar[-1]

datetime.date(2024, 8, 25)

In [5]:
hrFilenames = [x for x in dir_list if x.split("_")[0] == "heart"]
hrFilenames = [x for x in hrFilenames if pd.to_datetime(x[11:-5]).date() not in datesSoFar]
hrFilenames

['heart_rate-2023-02-05.json',
 'heart_rate-2023-11-14.json',
 'heart_rate-2024-08-26.json',
 'heart_rate-2023-09-13.json',
 'heart_rate-2022-10-12.json',
 'heart_rate-2023-04-13.json',
 'heart_rate-2024-08-27.json',
 'heart_rate-2024-02-07.json']

In [6]:
#takes like 40 minutes to make 15.8 million rows
import json

columnNames = ["sampleDT", "confidence", "value"]

samplesCount = 0
samplesList = []

for fn in hrFilenames:
    data = json.load(open(pathOfExport + exportFilePath + "/" + fn))
    for row in data:
        samplesList.append([pd.to_datetime(row["dateTime"] + "-0700"), row["value"]["confidence"], row["value"]["bpm"]])
        samplesCount += 1
        if samplesCount % 100_000 == 0:
            print(f"added {samplesCount} samples so far")
print(f"added {samplesCount} samples")

added 34326 samples


In [7]:
samplesList = sorted(samplesList, key=lambda x: x[0]) #sort by timestamp

In [8]:
import pandas as pd

fitbitHRdf = pd.DataFrame(data=samplesList, columns=columnNames)
fitbitHRdf = fitbitHRdf.set_index("sampleDT")
fitbitHRdf["confidence"] = fitbitHRdf["confidence"].astype('uint8')
fitbitHRdf["value"] = fitbitHRdf["value"].astype('uint8')
fitbitHRdf.dtypes

confidence    uint8
value         uint8
dtype: object

In [9]:
# concatenate the df's
# remove duplicate indexes
fitbitHRdf = pd.concat([dfSoFar, fitbitHRdf])
fitbitHRdf = fitbitHRdf[~fitbitHRdf.index.duplicated(keep="first")].sort_index()

In [10]:
# Get the size of the file in bytes
workingDataCompleteFile = "/home/chowder/Documents/workingData/fitbit/hr/fitbitHRdf.parquet.gzip"
fitbitHRdf.to_parquet(workingDataCompleteFile,
              compression='gzip') 

file_size = os.path.getsize(workingDataCompleteFile)
os.remove(workingDataCompleteFile)

In [11]:
import math
numFiles = math.ceil(file_size / (1024 * 1024 * 5))
rows_per_file = int(len(fitbitHRdf)/numFiles)

print(f"the file size of all the data is about {file_size // (1024 * 1024)} MB")
print(f"the total number of rows in the file is {len(fitbitHRdf)}")
print(f"splitting into {numFiles} number of files with {rows_per_file} rows per file")
print()

for fileNumber in range(numFiles + 1):
    startRow = fileNumber * rows_per_file
    if fileNumber == numFiles:
        endRow = len(fitbitHRdf) - 1
    else:
        endRow = ((fileNumber + 1) * rows_per_file) - 1

    print(f"saving rows {startRow} to {endRow}")
    
    parquetName = str(fitbitHRdf.iloc[startRow].name) +\
                  "_" +\
                  str(fitbitHRdf.iloc[endRow].name) +\
                  ".parquet.gzip"
    print(f"to a file named {parquetName}")

    fitbitHRdf.iloc[startRow:endRow+1].to_parquet(workingDataPath + parquetName,
              compression='gzip') 


the file size of all the data is about 86 MB
the total number of rows in the file is 15826297
splitting into 18 number of files with 879238 rows per file

saving rows 0 to 879237
to a file named 2020-05-13 17:02:34-07:00_2020-08-02 07:27:56-07:00.parquet.gzip
saving rows 879238 to 1758475
to a file named 2020-08-02 07:28:51-07:00_2020-10-22 19:11:11-07:00.parquet.gzip
saving rows 1758476 to 2637713
to a file named 2020-10-22 19:12:06-07:00_2021-01-11 15:28:18-07:00.parquet.gzip
saving rows 2637714 to 3516951
to a file named 2021-01-11 15:28:33-07:00_2021-03-31 17:02:38-07:00.parquet.gzip
saving rows 3516952 to 4396189
to a file named 2021-03-31 17:03:13-07:00_2021-06-18 16:06:46-07:00.parquet.gzip
saving rows 4396190 to 5275427
to a file named 2021-06-18 16:07:22-07:00_2021-09-06 21:51:09-07:00.parquet.gzip
saving rows 5275428 to 6154665
to a file named 2021-09-06 21:51:44-07:00_2021-11-30 20:39:36-07:00.parquet.gzip
saving rows 6154666 to 7033903
to a file named 2021-11-30 20:40:06-07